# How to use a team of Mosaia agents locally using Autogen

This Jupyter Notebook outlines the steps for importing a group of agents from Mosaia to be used in a local implementation of Autogen.

## Requirement

AutoBuild requires Python version 3.11 and `pyautogen[autobuild]`, which can be installed by the following command:

In [1]:
%pip install "pyautogen[autobuild]"

  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached FLAML-2.3.1-py3-none-any.whl.metadata (16 kB)
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached chromadb-0.5.16-py3-none-any.whl.metadata (6.8 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached pysqlite3-0.5.4-cp311-cp311-macosx_11_0_arm64.whl
  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp311-cp311-macosx_1

## Step 1: retrieve the autogen config json from Mosaia
Click the following link https://www.mosaia.ai/user/aaron/team/project-ideation-team?tab=agents in a browser. Click the blue `Export` on the top and then `autogen`. This will download an autogen config json to be used in this notebook. Make sure to have this file in the same directory location as this notebook.

## Step 2: import the required dependencies from autogen

In [2]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder
from autogen import UserProxyAgent, GroupChat, GroupChatManager

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## Step 2: prepare configuration and some useful functions
Prepare a `config_file_or_env` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable. A `default_llm_config` is also required for initialize the specific config of LLMs like seed, temperature, etc...

In [4]:
config_file_or_env = "OAI_CONFIG_LIST"
llm_config = {"temperature": 0}
config_list = autogen.config_list_from_json(config_file_or_env, filter_dict={"model": ["gpt-4o-mini"]})

## Step 3: create a AgentBuilder
Create a `AgentBuilder` with the specified `config_path_or_env`.

In [5]:
builder = AgentBuilder(config_file_or_env=config_file_or_env)

## Step 4: create agents using the autogen schema from Mosaia

reference the path to the autogen config json file on your local machine in the `builder.load` function

In [7]:
agent_list, agent_configs = builder.load(
    "./project-ideation-team_autogen_export.json"
)

Loading config from ./project-ideation-team_autogen_export.json
==> Creating agents...
Creating agent AI_Research_Expert...
Creating agent Product_Management_Expert...
Creating agent UX_Design_Expert...
Creating agent Market_Analysis_Expert...
Creating agent Business_Development_Expert...


## Step 4: create a group chat task runner
Using Autogen's `UserProxyAgent`, `GroupChat` and `GroupChatManager`. Create a task runner for the hydrated team of agents to work on an assigned task given by the `UserProxyAgent`.

In [8]:
def start_task(execution_task: str, agent_list: list, coding=True):
    user_proxy_system_message = "A human admin proxy for interacting with multi-agent systems"
    user_proxy = UserProxyAgent(
        name="user",
        code_execution_config={
            "work_dir": "coding",
            "use_docker": False,
        },
        system_message=user_proxy_system_message,
        is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
        human_input_mode="NEVER",
        default_auto_reply="TERMINATE",
        max_consecutive_auto_reply=1
    )

    agent_list.append(user_proxy)

    group_chat = GroupChat(
        agents=agent_list,
        messages=[],
        max_round=12,
        allow_repeat_speaker=agent_list[:-1] if coding is True else agent_list,
    )
    manager = GroupChatManager(
        groupchat=group_chat,
        llm_config={"config_list": config_list, **llm_config},
    )
    chat_result = user_proxy.initiate_chat(manager, message=execution_task)

    return chat_result

## Step 5: execute task
Let's run a task with our group of agents using the task runner

In [10]:
chat_result = start_task(
    execution_task="Give me some ideas for a viable AI startup",
    agent_list=agent_list,
)

user (to chat_manager):

Give me some ideas for a viable AI startup

--------------------------------------------------------------------------------

Next speaker: Market_Analysis_Expert

Market_Analysis_Expert (to chat_manager):

To generate viable AI startup ideas, we should consider current market trends, customer needs, and potential gaps in the market. Here are a few ideas:

1. **AI-Powered Personal Finance Assistant**: An application that uses AI to analyze users' spending habits, provide personalized budgeting advice, and suggest investment opportunities based on their financial goals.

2. **AI-Driven Health Monitoring System**: A platform that utilizes AI to analyze data from wearable devices, providing users with insights into their health metrics and suggesting lifestyle changes or alerting them to potential health issues.

3. **Smart Customer Support Chatbots**: Developing advanced chatbots that can handle complex customer inquiries, learn from interactions, and provide per

InternalServerError: Error code: 500 - {'error': {'message': 'The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.', 'type': 'model_error', 'param': None, 'code': None}}

## Step 6: view task results

In [ ]:
chat_result

## Step 7: clear the agent builder
Clear the cache of the agent builder to hydrate a new team of agents

In [ ]:
builder.clear_all_agents()